In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    gamma = individual[0]
    C = 1
    epsilon = 0.1

    if gamma <= 0 :
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,

Testing gamma range=(1e-07, 0.0001)...
MAPE terbaik untuk gamma range=(1e-07, 0.0001): 43.610520635072554
Testing gamma range=(1e-07, 0.001)...
MAPE terbaik untuk gamma range=(1e-07, 0.001): 43.425083733378145
Testing gamma range=(1e-07, 0.01)...
MAPE terbaik untuk gamma range=(1e-07, 0.01): 40.40063154450782
Testing gamma range=(1e-05, 0.0001)...
MAPE terbaik untuk gamma range=(1e-05, 0.0001): 43.609915342321685
Testing gamma range=(1e-05, 0.001)...
MAPE terbaik untuk gamma range=(1e-05, 0.001): 43.425688098430456
Testing gamma range=(1e-05, 0.01)...
MAPE terbaik untuk gamma range=(1e-05, 0.01): 40.301812756600945
Testing gamma range=(0.001, 0.1)...
MAPE terbaik untuk gamma range=(0.001, 0.1): 30.1871871106097
Testing gamma range=(0.001, 1)...
MAPE terbaik untuk gamma range=(0.001, 1): 27.84947015821664
Testing gamma range=(0.001, 5)...
MAPE terbaik untuk gamma range=(0.001, 5): 24.369518218162753

Rentang Gamma optimal: (0.001, 5)
MAPE terbaik: 24.369518218162753


In [5]:
def main():
    # Setup DEAP framework
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)
    
    toolbox = base.Toolbox()
    
    # Register attributes and constants
    toolbox.register("attr_float", random.uniform)
    
    # Register individual and population
    toolbox.register("individual", tools.initCycle, creator.Individual, 
                        (lambda: random.uniform(*gamma_range),), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
    toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
    toolbox.register("evaluate", evaluate)

    gamma_ranges = [(0.0000001, 0.0001), (0.0000001, 0.001), (0.0000001, 0.01), (0.00001, 0.0001), (0.00001, 0.001), (0.00001, 0.01), (0.001, 0.1), (0.001, 1), (0.001, 5)]
    crossover_rate = 0.1
    mutation_rate = 0.9
    population_size = 100
    num_generations = 75
    
    best_mape = float('inf')
    best_gamma_range = None
    
    results = []

    for gamma_range in gamma_ranges:        
        population = toolbox.population(n=population_size)

        for generation in range(num_generations):
            offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
            fits = toolbox.map(toolbox.evaluate, offspring)
            for fit, ind in zip(fits, offspring):
                ind.fitness.values = fit

            population = toolbox.select(offspring, k=len(population))

        best_individual = tools.selBest(population, k=1)[0]
        mape = best_individual.fitness.values[0]

        results.append((gamma_range, mape))
        print(f"MAPE terbaik untuk gamma range={gamma_range}: {mape}")

        if mape < best_mape:
            best_mape = mape
            best_gamma_range = gamma_range

    print(f"\nRentang Gamma optimal: {best_gamma_range}")
    print(f"MAPE terbaik: {best_mape}")
    
    # Plot hasil
    fig, ax = plt.subplots()
    gamma_ranges_str = [f"{r[0]}-{r[1]}" for r in gamma_ranges]
    mapes = [r[1] for r in results]
    ax.plot(gamma_ranges_str, mapes, marker='o')
    ax.set_xlabel('Gamma Range')
    ax.set_ylabel('MAPE')
    ax.set_title('MAPE vs Gamma Range')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.show()

if __name__ == "__main__":
    main()